In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
pip install lxml BeautifulSoup4 

Note: you may need to restart the kernel to use updated packages.


In [13]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [14]:
ny_crimes=pd.read_csv('https://data.cityofnewyork.us/resource/qgea-i56i.csv')

In [18]:
ny_crimes.head()

,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,pd_desc,crm_atpt_cptd_cd,law_cat_cd,boro_nm,loc_of_occur_desc,prem_typ_desc,juris_desc,jurisdiction_code,parks_nm,hadevelopt,housing_psa,x_coord_cd,y_coord_cd,susp_age_group,susp_race,susp_sex,transit_district,latitude,longitude,lat_lon,patrol_boro,station_name,vic_age_group,vic_race,vic_sex
0,325341655,2015-02-11T00:00:00.000,15:00:00,NaN,NaN,73,2015-02-11T00:00:00.000,359,OFFENSES AGAINST PUBLIC ADMINI,749.0,VIOLATION OF ORDER OF PROTECTI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,TILDEN,405.0,1009656.0,181287.0,25-44,BLACK,M,NaN,40.664239,-73.908425,"(40.664239422, -73.908425011)",PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M
1,393816841,2012-03-17T00:00:00.000,10:30:00,2012-03-17T00:00:00.000,11:00:00,69,2012-03-17T00:00:00.000,344,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1014035.0,174133.0,NaN,NaN,NaN,NaN,40.644590,-73.892672,"(40.644589618, -73.892672426)",PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F
2,802896158,2016-10-27T00:00:00.000,13:48:00,2016-11-03T00:00:00.000,13:49:00,71,2016-11-03T00:00:00.000,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,BROOKLYN,INSIDE,PUBLIC SCHOOL,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1000222.0,179282.0,<18,BLACK,M,NaN,40.658758,-73.942435,"(40.658758183, -73.942434788)",PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M
3,633812343,2014-11-27T00:00:00.000,19:00:00,2014-11-27T00:00:00.000,22:30:00,112,2014-11-28T00:00:00.000,104,RAPE,157.0,RAPE 1,COMPLETED,FELONY,QUEENS,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1025420.0,202485.0,25-44,WHITE HISPANIC,M,NaN,40.722364,-73.851474,"(40.722363687, -73.851473894)",PATROL BORO QUEENS NORTH,NaN,25-44,WHITE,F
4,300349533,2013-12-11T00:00:00.000,13:30:00,2013-12-11T00:00:00.000,14:15:00,24,2013-12-12T00:00:00.000,109,GRAND LARCENY,438.0,"LARCENY,GRAND FROM BUILDING (NON-RESIDENCE) UN...",COMPLETED,FELONY,MANHATTAN,INSIDE,DOCTOR/DENTIST OFFICE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,992848.0,228356.0,NaN,NaN,NaN,NaN,40.793465,-73.968950,"(40.793464597, -73.968949638)",PATROL BORO MAN NORTH,NaN,45-64,WHITE,F


In [20]:
df=ny_crimes.drop(['cmplnt_num','cmplnt_fr_dt','cmplnt_fr_tm','cmplnt_to_dt','cmplnt_to_tm','addr_pct_cd','rpt_dt','ky_cd','crm_atpt_cptd_cd','loc_of_occur_desc'],axis=1)

In [21]:
df.head()

,ofns_desc,pd_cd,pd_desc,law_cat_cd,boro_nm,prem_typ_desc,juris_desc,jurisdiction_code,parks_nm,hadevelopt,housing_psa,x_coord_cd,y_coord_cd,susp_age_group,susp_race,susp_sex,transit_district,latitude,longitude,lat_lon,patrol_boro,station_name,vic_age_group,vic_race,vic_sex
0,OFFENSES AGAINST PUBLIC ADMINI,749.0,VIOLATION OF ORDER OF PROTECTI,MISDEMEANOR,BROOKLYN,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,TILDEN,405.0,1009656.0,181287.0,25-44,BLACK,M,NaN,40.664239,-73.908425,"(40.664239422, -73.908425011)",PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M
1,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,MISDEMEANOR,BROOKLYN,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1014035.0,174133.0,NaN,NaN,NaN,NaN,40.644590,-73.892672,"(40.644589618, -73.892672426)",PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F
2,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",VIOLATION,BROOKLYN,PUBLIC SCHOOL,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1000222.0,179282.0,<18,BLACK,M,NaN,40.658758,-73.942435,"(40.658758183, -73.942434788)",PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M
3,RAPE,157.0,RAPE 1,FELONY,QUEENS,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1025420.0,202485.0,25-44,WHITE HISPANIC,M,NaN,40.722364,-73.851474,"(40.722363687, -73.851473894)",PATROL BORO QUEENS NORTH,NaN,25-44,WHITE,F
4,GRAND LARCENY,438.0,"LARCENY,GRAND FROM BUILDING (NON-RESIDENCE) UN...",FELONY,MANHATTAN,DOCTOR/DENTIST OFFICE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,992848.0,228356.0,NaN,NaN,NaN,NaN,40.793465,-73.968950,"(40.793464597, -73.968949638)",PATROL BORO MAN NORTH,NaN,45-64,WHITE,F


In [31]:
limit=100
df_ny1=df.iloc[0:limit,:]
df_ny1.shape

(100, 25)

In [32]:
#downloading San Francisco Dataset

In [33]:
df_sf = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DV0101EN/labs/Data_Files/Police_Department_Incidents_-_Previous_Year__2016_.csv')

In [34]:
limit = 100
df_sf = df_incidents.iloc[0:limit, :]

In [35]:
df_sf.shape

(100, 13)

In [36]:
latitude_sf = 37.77
longitude_sf = -122.42

In [65]:
latitude_ny=40.7697281
longitude_ny=-74.0060152

In [66]:
ny_map = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=11.5)
ny_map

In [70]:
# instantiate a feature group for the incidents in the dataframe
incidents_ny = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_ny1.latitude, df_ny1.longitude):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
ny_map.add_child(incidents_ny)

In [76]:
sanfran_map = folium.Map(location=[latitude_sf, longitude_sf], zoom_start=12)
sanfran_map

In [81]:
incidents_sf = folium.map.FeatureGroup()
for lat, lng, in zip(df_sf.Y, df_sf.X):
    incidents_sf.add_child(folium.features.CircleMarker([lat, lng],radius=5,color='yellow',fill=True,fill_color='blue',fill_opacity=0.6))

# add incidents to map
sanfran_map.add_child(incidents_sf)